## Quantum Kernels that Exploit Structure in Data

An important approach in the search for practical quantum advantage in machine learning is to identify quantum kernels for learning problems that exploit underlying structure in the data. A class of quantum kernels were introduced in Ref. [1] that can exploit group structure in data. Such kernels are called _covariant quantum kernels_, which we will explore here.

### Covariant Quantum Kernels

Let's consider a classification problem for a dataset with samples $x$ in a data space $\mathcal{X}$ and binary labels $y=\pm1$. Here, the dataspace will correspond to a group $G$. Examples of learning problems for data with group structure could include learning permutations or classifying translations. A sample $x$ is encoded in the quantum-enhanced feature space through a unitary representation $D_x$ and a fiducial (reference) state $|\psi\rangle=V|0^{n}\rangle$, producing the feature state:

$$
\Phi(x)=D_{x}|\psi\rangle\langle\psi|D_{x}^{\dagger}=D_{x}V|0^n\rangle\langle0^n|V^{\dagger}D_{x}^{\dagger}.
$$

The kernel entry for two samples $x,\tilde{x}$ is computed from the Hilbert-Schmidt inner product of the two quantum feature states, which can be viewed as the transition amplitude,

$$
K(x,\tilde{x})=|\langle0^n|V^{\dagger}D^{\dagger}_{x}D_{\tilde{x}}V|0^n\rangle|^2.\qquad(1)
$$

This quantity can be estimated on a quantum computer by preparing the state $V^{\dagger}D^{\dagger}_{x}D_{\tilde{x}}V|0^n\rangle$ and measuring the frequency of the all-zero outcome [2,3].

In general, the choice of the fiducial state ![equation](https://latex.codecogs.com/svg.latex?\inline&space;|\psi\rangle) is not known _a priori_ and can significantly impact the performance of the classifier. Quantum kernel training (QKT) is one method we can use to find a good fiducial state for a given group.

### Aligning Covariant Quantum Kernels on a Dataset

In practice, SVMs involve a choice of the kernel function. Sometimes, symmetries in the data can inform this selection, other times it is chosen in an ad hoc manner. Kernel alignment is an approach to learning a kernel on a given dataset by iteratively adapting it to have high similarity to a target kernel informed from the underlying data distribution. As a result, the SVM with an aligned kernel will likely generalize better to new data than with an unaligned kernel [4]. Using this concept, Ref. [1] introduced an algorithm for quantum kernel alignment, which provides a way to learn a quantum kernel from a family of kernels. Specifically, the algorithm optimizes the parameters in a quantum circuit to maximize the alignment of the corresponding quantum kernel while converging to the maximum SVM margin. More information about QKA can be found in the [background material](https://github.com/IBM-Quantum-Prototypes/quantum-kernel-training/blob/main/docs/background/svm_weighted_kernel_alignment.ipynb).

Applying QKA to covariant quantum kernels corresponds to extending Eq. (1) to

$$
K_{\lambda}(x,\tilde{x})=|\langle0^n|V^{\dagger}_{\lambda}D^{\dagger}_{x}D_{\tilde{x}}V_\lambda|0^n\rangle|^2\qquad(2)
$$

where the circuit preparing the fiducial state is now parametrized by $\lambda$. Then, the parameters of the circuit can be trained on a given dataset according to the QKA procedure.

### Covariant Quantum Kernels for a Learning Problem

Let's take a closer look at covariant quantum kernels and QKA by considering a specific learning problem called _labeling cosets with error_ [1].  We will briefly describe the learning problem, show how the datasets are generated, and then choose a parametrized fiducial state to train with QKA. 

#### Define the Learning Problem
In this binary classification problem, we use a group and a subgroup to form two cosets, which will represent our two data classes [5]. We take the group $G=SU(2)^{\otimes{n}}$ for $n$ qubits, which is the special unitary group of $2\times2$ matrices and has wide applicability in nature, for example, the Standard Model of particle physics and in many condensed matter systems. We take the (graph-stabilizer) subgroup $S_{\mathrm{graph}}<{G}$ with $S_{\mathrm{graph}}=\langle{\{}X_i\otimes_{k:(k,i)\in\mathcal{E}}Z_k{\}}_{i\in\mathcal{V}}\rangle$ for a graph with edges $\mathcal{E}$ and vertices $\mathcal{V}$. Note that the stabilizers fix a stabilizer state such that $D_s|\psi\rangle=|\psi\rangle$, $\forall s\in S_{\mathrm{graph}}$. This observation will be useful later on when we choose a fiducial state. Finally, we define two left-cosets $C_{\pm}=c_{\pm}S_{\mathrm{graph}}$ by drawing two $c_{\pm}\in{G}$ at random.

#### Generate the Datasets
Now that we've defined the cosets, we generate the dataset by writing the rotations of the group $G$ as

$$D(\theta_1,\theta_2,\theta_3=0)=\exp(i\theta_1X)\exp(i\theta_2Z)\in{SU(2)},$$ 

so that each qubit is parametrized by the first two Euler angles (the third we set to zero). Then, we draw randomly two sets of angles $c_{\pm}\in[-\pi/2,\pi/2]^{2n}$ for an $n$-qubit problem, which define the two left-cosets (corresponding to the two classes). Combining these representative angles with the angles of the stabilizer group elements, the elements of the cosets can again be written in terms of Euler angles: $D_{\theta}=\otimes_{k=1}^{n}D(\theta_{2k-1},\theta_{2k},0)$. Thus, we express every element of a coset by the Euler angles $\theta=(\theta_1,\theta_2,\ldots,\theta_{2n-1},\theta_{2n})$. We build training and testing sets by randomly drawing elements from the cosets $C_{\pm}$. That is, each data sample is a list of $2n$ angles and the labels $y=\pm1$ indicate to which coset the sample belongs.

#### Choose a Fiducial State
Next, we select a fiducial state. A natural candidate is the stabilizer state we encountered above. Why? Because this is a subgroup invariant state, $D_s|\psi\rangle=|\psi\rangle$, which causes the data for a given coset to be mapped to a unique state: $D_{c_{\pm}}D_s|\psi\rangle=D_{c_{\pm}}|\psi\rangle$, $\forall{s}\in{S_{\mathrm{graph}}}$. This means the classifier only needs to distinguish the _two_ states $\Phi(c_{\pm})=D_{c_{\pm}}|\psi\rangle\langle\psi|D^{\dagger}_{c_{\pm}}$ for every element of the coset. At this point, the problem we've described is noise free. We can add a normal random error of variance 0.01 to the Euler angles of the dataset. This noise will perturb the two states $\Phi(c_{\pm})$, but if the variance is sufficiently small, we expect the states will still be classified correctly. Using what we learned about the subgroup invariant state, let's construct our fiducial state as a parametrized stabilizer state associated with the coupling graph $(\mathcal{E},\mathcal{V})$ given by the device connectivity. Then, we can use QKA to find its optimal parameters. Specifically, we'll replace the initial layers of Hadamards in the graph state with rotations $R_Y(\lambda)=\exp(-i(\lambda/2)Y)$ by an angle $\lambda$:

$$
|\psi_\lambda\rangle=V_\lambda|0^n\rangle=\prod_{(k,t)\in\mathcal{E}}CZ_{k,t}\prod_{k\in\mathcal{V}}R_{Y_k}(\lambda),\qquad(3)
$$

where $CZ=\mathrm{diag}(1,1,1,-1)$. If we initialize the kernel with $\lambda\approx0$, we expect the classifier to perform poorly on a test set. However, from this starting point, the quantum kernel alignment algorithm should converge towards the optimal $\lambda=\pi/2$ (recovering the Hadamards of the graph state) and the classifier should yield 100% test accuracy.

### Dataset Examples

In this toolkit, we've defined two specific problem instances (and their [datasets](https://github.com/IBM-Quantum-Prototypes/quantum-kernel-training/tree/main/data)) to test these ideas out. To do this, we choose a particular graph for the subgroup, and then generate the corresponding datasets as described above. Let's consider the quantum device `ibmq_montreal`, with coupling map shown below:

<p align="center">
  <img src="../../docs/images/chip.png" width="505">
</p>


We'll pick two different subgraphs, one for [7 qubits](https://github.com/IBM-Quantum-Prototypes/quantum-kernel-training/blob/main/data/dataset_graph7.csv) and one for [10 qubits](https://github.com/IBM-Quantum-Prototypes/quantum-kernel-training/blob/main/data/dataset_graph10.csv), to define our problem instances. Then, using the parametrized fiducial state in Eq. (3), we train the quantum kernel with QKA to learn a set of parameters.

<p align="center">
  <img src="../../docs/images/subgraphs.png" width="580">
</p>


### References

[1] J. R. Glick, T. P. Gujarati, A. D. Córcoles, Y. Kim, A. Kandala, J. M. Gambetta, K. Temme, arXiv:2105.03406 (2021) [link](https://arxiv.org/abs/2105.03406)<br>
[2] V. Havlíček, A. D. Córcoles, K. Temme, A. W. Harrow, A. Kandala, J. M. Chow, and J. M. Gambetta, Nature 567, 209-212 (2019) [link](https://doi.org/10.1038/s41586-019-0980-2) <br>
[3] Y. Liu, S. Arunachalam, and K. Temme, arXiv:2010.02174 (2020) [link](https://arxiv.org/abs/2010.02174) <br>
[4] N. Cristianini, J. Shawe-Taylor, A. Elisseeff, and J. Kandola, Advances in Neural Information Processing Systems 14 (2001) [link](https://proceedings.neurips.cc/paper/2001/file/1f71e393b3809197ed66df836fe833e5-Paper.pdf) <br>
[5] J. Rotman, An Introduction to the Theory of Groups (1995) [link](https://link.springer.com/book/10.1007/978-1-4612-4176-8)<br>

Qiskit Global Summer School on Quantum Machine Learning, lecture [6.3](https://learn.qiskit.org/summer-school/2021/lec7-1-quantum-kernels-practice), Quantum Kernels in Practice